<a href="https://colab.research.google.com/github/sunandhini96/TSAI_ERAV1/blob/main/Capstone/Stage_1/stage_1_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/capstone

/content/drive/MyDrive/capstone


In [ ]:
# import os
# import zipfile

# # Directory containing the ZIP files
# zip_dir = '/content/drive/MyDrive/capstone/'

# # Iterate over the files in the directory
# for file_name in os.listdir(zip_dir):
#     if file_name.endswith('.zip'):  # Check if the file is a ZIP file
#         file_path = os.path.join(zip_dir, file_name)
#         output_dir = zip_dir  # Extract to the same directory
#         with zipfile.ZipFile(file_path, 'r') as zip_ref:
#             zip_ref.extractall(output_dir)
#             print(f"Extracted {file_name} to {output_dir}")


In [ ]:
# import os

# # Directory containing the files
# directory = '/content/drive/MyDrive/capstone/train2017/'

# # Get the list of files in the directory
# files = os.listdir(directory)

# # Count the number of files
# num_files = len(files)

# print(f'Number of files in directory: {num_files}')


In [3]:
!git clone https://github.com/sunandhini96/TSAI_ERAV1.git

fatal: destination path 'TSAI_ERAV1' already exists and is not an empty directory.


In [4]:
cd /content/drive/MyDrive/capstone/TSAI_ERAV1/Capstone/Stage_1

/content/drive/MyDrive/capstone/TSAI_ERAV1/Capstone/Stage_1


In [5]:
!pip install -r requirements.txt

  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-ziezvb53
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-ziezvb53
  Resolved https://github.com/m-bain/whisperx.git to commit 8227807fa9e076901ea4b4fbbf79c9777a6f5e03
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/SYSTRAN/faster-whisper.git (to revision 0.10.0) to /tmp/pip-install-1f3fyy8d/faster-whisper_e02366c02f244574a0ca56cd883d0a96
  Running command git clone --filter=blob:none --quiet https://github.com/SYSTRAN/faster-whisper.git /tmp/pip-install-1f3fyy8d/faster-whisper_e02366c02f244574a0ca56cd883d0a96
  Running command git checkout -q e1a218fab1ab02d637b79565995bf1a9c4c83a09
  Resolved https://github.com/SYSTRAN/faster-whisper.git to commit e1a218fab1ab02d637b79565995bf1a9c4c83a09
  Preparing metadata (setup.py) ... done


In [6]:
!pip install glob2

In [7]:
import glob2
import torch
from network import CLIPPhi2Model, train_model, frange_cycle_linear
from dataset import collate_fn, llavadataset
from torch.utils.data import random_split, DataLoader
import wandb
import torch.nn as nn
from transformers import AutoTokenizer
import pickle
#import bitsandbytes as bnb
import gc


def main():
    with open("/content/drive/MyDrive/capstone/coco_captions.pickle", "rb") as fp:   # Unpickling
        coco_unpickle = pickle.load(fp)

    clip_model_name = "wkcn/TinyCLIP-ViT-61M-32-Text-29M-LAION400M"
    phi_model_name  = "microsoft/phi-2"
    train_batch_size = 2
    val_batch_size   = 2
    device     = 'cuda'
    tokenizer  = AutoTokenizer.from_pretrained(phi_model_name, trust_remote_code=True)

    # model
    MModalGPT        = CLIPPhi2Model().to(device)
    max_steps        = 10000
    model_save_step  = 100
    model_val_step   = 1000
    log_step         = 100
    max_token_filter = 35

    # data loaders
    train_dataloader = DataLoader(llavadataset(coco_unpickle, phi_model_name,clip_model_name,'train',tokenizer),
                      collate_fn=collate_fn, batch_size=train_batch_size, num_workers = 10, shuffle=True, pin_memory=True)

    val_dataloader   = DataLoader(llavadataset(coco_unpickle, phi_model_name,clip_model_name,'val',tokenizer),
                      collate_fn=collate_fn, batch_size=val_batch_size, num_workers = 10, shuffle=True, pin_memory=True)

    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, MModalGPT.parameters()), lr=1e-4)
    train_model(MModalGPT, train_dataloader, val_dataloader, optimizer, device, max_steps,model_save_step,model_val_step,log_step,max_token_filter,tokenizer)

import wandb

wandb.login(key="496133cd12c3c9c1f19d8a2cfeb8986ddc1d93ec")

wandb: Currently logged in as: gsunandhini. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
if __name__ == "__main__":
    wandb.init(project="tsai_multimodal_gpt_project", name="step1_projector_pretrain") #,log_model='all')
    torch.cuda.amp.autocast(enabled=True)
    torch.cuda.empty_cache()
    gc.collect()
    #torch.set_float32_matmul_precision('medium')
    main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Train size 532577 and validation size 59176
Train size 532577 and validation size 59176
Training started.
Step 0/10000: Avg Running Loss = 7.805176734924316
Saving Checkpoint for step :  100
Step 100/10000: Avg Running Loss = 6.727888708114624
Saving Checkpoint for step :  200
Step 200/10000: Avg Running Loss = 5.945052621364593
Saving Checkpoint for step :  300
Step 300/10000: Avg Running Loss = 5.669799976348877
Saving Checkpoint for step :  400
Step 400/10000: Avg Running Loss = 5.50166642665863
Saving Checkpoint for step :  500
Step 500/10000: Avg Running Loss = 5.4119014954566955
Saving Checkpoint for step :  600
Step 600/10000: Avg Running Loss = 5.457903873920441
Saving Checkpoint for step :  700
Step 700/10000: Avg Running Loss = 5.437376885414124
Saving Checkpoint for step :  800
Step 800/10000: Avg Running Loss = 5.254926381111145
Saving Checkpoint for step :  900
Step 900/10000: Avg Running Loss = 5.15674718618393
Saving Checkpoint for step :  1000
0 - Target captions:
 Peop